In [1]:
!rm -Rf HMP_Dataset
!git clone https://github.com/wchill/HMP_Dataset

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20190831124229-0000
KERNEL_ID = 09289198-fec6-4e04-a84b-c48131496234
Cloning into 'HMP_Dataset'...
remote: Enumerating objects: 865, done.
remote: Total 865 (delta 0), reused 0 (delta 0), pack-reused 865
Receiving objects: 100% (865/865), 1010.96 KiB | 4.91 MiB/s, done.
Checking out files: 100% (848/848), done.


In [2]:
!ls HMP_Dataset/Brush_teeth

Accelerometer-2011-04-11-13-28-18-brush_teeth-f1.txt
Accelerometer-2011-04-11-13-29-54-brush_teeth-f1.txt
Accelerometer-2011-05-30-08-35-11-brush_teeth-f1.txt
Accelerometer-2011-05-30-09-36-50-brush_teeth-f1.txt
Accelerometer-2011-05-30-10-34-16-brush_teeth-m1.txt
Accelerometer-2011-05-30-21-10-57-brush_teeth-f1.txt
Accelerometer-2011-05-30-21-55-04-brush_teeth-m2.txt
Accelerometer-2011-05-31-15-16-47-brush_teeth-f1.txt
Accelerometer-2011-06-02-10-42-22-brush_teeth-f1.txt
Accelerometer-2011-06-02-10-45-50-brush_teeth-f1.txt
Accelerometer-2011-06-06-10-45-27-brush_teeth-f1.txt
Accelerometer-2011-06-06-10-48-05-brush_teeth-f1.txt


In [3]:
from pyspark.sql.types import StructType, StructField, IntegerType

schema = StructType([
    StructField("x", IntegerType(), True),
    StructField("y", IntegerType(), True),
    StructField("z", IntegerType(), True)])

In [ ]:
import os
import fnmatch

d = 'HMP_Dataset/'

# filter list for all folders containing data (folders that don't start with .)
file_list_filtered = [s for s in os.listdir(d) if os.path.isdir(os.path.join(d,s)) & ~fnmatch.fnmatch(s, '.*')]

from pyspark.sql.functions import lit

#create pandas data frame for all the data

df = None

for category in file_list_filtered:
    data_files = os.listdir('HMP_Dataset/'+category)
    
    #create a temporary pandas data frame for each data file
    for data_file in data_files:
        print(data_file)
        temp_df = spark.read.option("header", "true").option("header", "false").option("delimiter", " ").csv('HMP_Dataset/'+category+'/'+data_file,schema=schema)
        
        #create a column called "source" storing the current CSV file
        temp_df = temp_df.withColumn("source", lit(data_file))
        
        #create a column called "class" storing the current data folder
        temp_df = temp_df.withColumn("class", lit(category))
        
        #append to existing data frame list
        #data_frames = data_frames + [temp_df]
                                                                                                             
        if df is None:
            df = temp_df
        else:
            df = df.union(temp_df)
        


Accelerometer-2011-04-11-13-28-18-brush_teeth-f1.txt
Accelerometer-2011-04-11-13-29-54-brush_teeth-f1.txt
Accelerometer-2011-05-30-08-35-11-brush_teeth-f1.txt
Accelerometer-2011-05-30-09-36-50-brush_teeth-f1.txt
Accelerometer-2011-05-30-10-34-16-brush_teeth-m1.txt
Accelerometer-2011-05-30-21-10-57-brush_teeth-f1.txt
Accelerometer-2011-05-30-21-55-04-brush_teeth-m2.txt
Accelerometer-2011-05-31-15-16-47-brush_teeth-f1.txt
Accelerometer-2011-06-02-10-42-22-brush_teeth-f1.txt
Accelerometer-2011-06-02-10-45-50-brush_teeth-f1.txt
Accelerometer-2011-06-06-10-45-27-brush_teeth-f1.txt
Accelerometer-2011-06-06-10-48-05-brush_teeth-f1.txt
Accelerometer-2011-03-24-10-24-39-climb_stairs-f1.txt
Accelerometer-2011-03-24-10-25-44-climb_stairs-f1.txt
Accelerometer-2011-03-29-09-55-46-climb_stairs-f1.txt
Accelerometer-2011-04-05-18-21-22-climb_stairs-f1.txt
Accelerometer-2011-04-05-18-32-29-climb_stairs-f1.txt
Accelerometer-2011-04-11-11-44-35-climb_stairs-f1.txt
Accelerometer-2011-04-11-11-57-50-climb_

In [ ]:
df.write.parquet('hmp.parquet')
